In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as pt
import math as ma
import scipy as sc
## All Checked

In [2]:
# list_ex_1=[np.array([i for i in range(5)]),np.array([i**2 for i in range(3)])]
# print(list_ex_1)
# list_ex_1[2]=np.array([2,7,8,4])
# print(list_ex_1)

# show_img        
To load the image
# pad_img         
To pad the image

# Relu            
Relu function used
# softmax         
Softmax activation
# activation      
The function applying it on the array

# c_para_make     
Making a list of all nested arrays containing the parameters to learn
# f_para_make     
Making a list of all nested arrays containing the parameters to learn

# nn_make         
Making a list of all nested arrays containing the parameters to learn, a list of all the modified data arrays

# nn_struc_f      
Making a list of all nested arrays containing the known matrices  
# nn_struc_c      
Making a list of all the nested arrays containing the known matrices 

# conv            a
# max_pol         a
# conv_tr_layer   a
# nn_fwd_layer    a

# forward_prop    a
# ack_prop        a

In [3]:
# Converting the dataset in the required form
global row_pixel, col_pixel

df_base=pd.read_csv('train.csv')
df=np.array(df_base.iloc[:,1:])
target=np.array(df_base.iloc[:,0])
df_sh=df.shape
row_pixel=col_pixel=int(ma.sqrt(df_sh[1]))
df=df.reshape(df_sh[0],row_pixel,col_pixel)
df_sh=df.shape
m=df.shape[0]
#  m is int not ndarray int type
## All Checked

In [4]:
output=np.array([i for i in range(10)])
# to print an image
def show_img(array):
    pt.imshow(array,cmap='gray')
    pt.show()
def pad_img(array,p):
    ar=array.copy()
    n=row_pixel
    h_pad_0=np.zeros((n,1), dtype=int)
    y=np.hstack([h_pad_0]*p)
    x=np.hstack((y,array,y))
    v_pad_1=np.zeros((1,n+2*p), dtype=int)
    y=np.vstack([v_pad_1]*p)
    x=np.vstack((y,x,y))
    return x
# a_1=pad_img(df[1],2)
# print(a_1.shape)

In [5]:
# Example of nn_structure variable
nn_struc_c=[[3,(3,3),1,(2,2),2,'Relu'],[1,(3,3),1,(2,2),1,'sigmoid']]
# ---------_^
# Layers marked
# format of variables (below)
# 'ReLu','sigmoid','tanh','softmax'
# [no. of layers->[filters,(filter_row,filter_row),padding,(pol_filter_row,pol_filter_row),pol_stride,func]...]
print(nn_struc_c)

"""
A single layer's information as we want to get the idea of convolution layers
For feature layers we define a separate structure  
"""

nn_struc_f=[100,40,10]

[[3, (3, 3), 1, (2, 2), 2], [1, (3, 3), 1, (2, 2), 1]]


In [6]:
def bytesize(lis):
    l=len(lis)
    c=0
    for i in range(l):
        c+=lis[i].size*lis[i].itemsize
    return c

In [7]:
# To generate the list of array of parameters for all convolution layers
def c_para_constr(rows,nn_struc_c):
    row_iter=rows
    c_layers=len(nn_struc_c)
    c_wt_list=[]
    c_b_list=[]
    c_pol_list=[]
    for i in range(c_layers):
        filters=nn_struc_c[i][0]
        conv_filter_row=nn_struc_c[i][1][0]
        padding=nn_struc_c[i][2]
        pol_stride=nn_struc_c[i][4]
        new_row=ma.floor((row_iter-conv_filter_row+2*padding)/pol_stride)+1
        c_wt_list.append(np.random.rand(filters,new_row,new_row))
        c_b_list.append(np.random.rand(filters,new_row))
        # pol_element=np.array([nn_struc_c[i][3],nn_struc_c[i][4]])
        # c_pol_list.append(pol_element)
        row_iter=new_row
    total_features=c_b_list[-1].size
    return c_wt_list,c_b_list,total_features#,c_pol_list

# To generate the list of array of parameters for all feature layers 
def f_para_constr(nn_struc_f,init_features):
    f_wt_list=[]
    f_b_list=[]
    features=init_features
    f_layers=len(nn_struc_f)
    for i in range(f_layers):
        f_wt_list.append(np.random.rand(nn_struc_f[i],features))
        f_b_list.append(np.random.rand(features,1))
        features=nn_struc_f[i]
    return f_wt_list,f_b_list


In [8]:
def nn_make(nn_struc_c,nn_struc_f,c_b,f_b):
    list_c=[]
    ar_f=np.array(nn_struc_f)
    list_all=[]
    layer_c=len(c_b)
    layer_f=len(f_b)
    n=row_pixel
    for i in range (layer_c):
        element=[]
        ax_0=nn_struc_c[i][0]
        c_row=nn_struc_c[i][1][0]
        padding=nn_struc_c[i][2]
        pol_row=nn_struc_c[i][3][0]
        pol_stride=nn_struc_c[i][4]
        ax_1=ma.floor((n-c_row+2*padding)/pol_stride)+1
        new_dim_c=(ax_0,ax_1,ax_1)
        ax_1=ma.floor((ax_1-pol_row)/pol_stride)+1
        new_dim_p=(ax_0,ax_1,ax_1)
        element.append(new_dim_c)
        element.append(new_dim_p)
        list_c.append(element)
        n=ax_1
    init_features=element[1].size
    list_all.append(list_c)
    list_all.append(init_features)
    list_all.append(ar_f)
    return list_all

In [9]:
# a_001,a_002,a_003=c_para_constr(row_pixel,nn_struc_c)
# print(a_001,a_002,a_003)
# print(bytesize(a_001)+bytesize(a_002))

In [10]:
# Rectified Linear Unit
def ReLu(array):
    array_0=np.zeros(array.shape,dtype=float)    
    array=np.maximum(array,array_0)
    return array

def softmax(array):
    array=np.exp(array)
    array=array/(np.sum(array))
    return array

# For the activation
def activation(array,func):
    if func=='ReLu':
        array=ReLu(array)
    elif func=='sigmoid':
        array=1/(1+np.exp((-1)*array))
    elif func=='tanh':
        array=np.tanh(array)
    elif func=='softmax':
        array=softmax(array)
    return array


In [ ]:

# For the cost function
def cost_array(expected,label):
    cost=(label-expected)**2
    return cost_array

# For the derivatives
def p_diff_cust(variable,func):
    if func=='cost':
        derivative=2*np.sqrt(variable)
    elif func=='ReLu':
        derivative=variable>0
        derivative+=0
    elif func=='sigmoid':
        derivative=variable-variable**2
    elif func=='tanh':
        derivative=1-variable**2
    elif func=='softmax':
        derivative=variable-variable**2
    return derivative

def learn_part():

    return

# For the calculation of gradient for an entire layer 
def calc_grad_c_layer():

    return

In [11]:
# Convolution Layer
def conv(dat,layer_info,c_wt_layer,c_b_layer):
    no_filters=layer_info[0]
    r=layer_info[1]
    padding=layer_info[2]
    n=row_pixel
    dat_pad=pad_img(dat,layer_info[2])
    nn_sample=np.vstack([dat]*no_filters)
    temp_conv_row=n-r+1
    ar=np.zeros((no_filters,temp_conv_row,temp_conv_row),dtype=float)
    for i in range(r):
        for j in range(r):
            t=(n-r+i+1)
            u=(n-r+j+1)
            # bradcast might be needed of c_wt_layer
            ar[:,i:t:r,j:t:r]=c_wt_layer[:,i,j]*nn_sample[:,i:t:r,j:t:r]
    c_b_array=np.array([np.full_like(ar,c_b_layer[i],dtype=float)])
    ar=ar+c_b_layer
    return ar

# Pooling layer
def max_pol(ar,layer_info):
    n=row_pixel
    pol_row=layer_info[3][0]
    stride=layer_info[4]
    temp_row=ma.floor((n-pol_row)/stride)+1
    pol=np.zeros(np.shape(ar[:,:temp_row:stride,:temp_row:stride]),dtype=float)
    for i in range (pol_row):
        for j in range (pol_row):
            pol=np.maximum(ar[:,i:(temp_row+i):stride,j:(temp_row+j):stride],pol)
    return pol

# Transforming laer consisting both conv() and max_pol() 
def conv_tr_layer(dat,layer_info,func,c_wt_layer,c_b_layer,list_all_layer):
    ar_01=dat.copy()
    ar_01=conv(ar,layer_info,c_wt_layer,c_b_layer)
    list_all_layer[0]=ar
    ar=max_pol(ar,layer_info)
    ar=activation(ar,func)
    list_all_layer[1]=ar
    return ar

# For forward neural network propagation over a single layer
def nn_fwd_layer(prev,num_new,func,f_wt_layer,f_b_layer):
    new=np.add((f_wt_layer@prev),f_b_layer)
    new=activation(new,func)
    return new

In [12]:
# For forward propagation over decided layers
def forward_prop(dat,c_wt_list,c_b_list,nn_struc_c,nn_struc_f,list_all):
    l_c=len(nn_struc_c)
    ar=dat.copy()
    for i in range(l_c):
        new_ar=conv_tr_layer(ar,c_wt_list[i],c_b_list[i],list_all[i])
        ar=new_ar
    init_fc_nodes=list_all[1]
    l_f=len(nn_struc_f)
    for i in range(l_f):
        final_fc_nodes=nn_struc_f[i]
        new_ar=nn_fwd_layer(init_fc_nodes,final_fc_nodes,list_all[2][i][0],list_all[2][i][1])
    return ar 

In [ ]:
# For backward neural network propagation
def back_prop():
    
    return

In [13]:
################################################### multi dimensional matrix multiplication works when used with 
# x=np.array([i for i in range(9)]).reshape(3,3)
# print(x)
# y=np.vstack([x]*3)
# print(y)
# y=y.reshape(3,3,3)
# print(y)
# z=np.array([12,13,14]).reshape(3,1)
# print(z)
# print(y+z)
# ansxz=x@z
# print(ansxz)
# z=np.vstack([z]*3).reshape(3,3,1)
# print(z)
# print(y@z)

In [17]:
# x_exp=np.array([i for i in range(9)]).reshape(3,3)
# y_exp=x_exp>0
# print(y_exp)
# z_exp=y_exp+0
# print(z_exp)

[[False  True  True]
 [ True  True  True]
 [ True  True  True]]
[[0 1 1]
 [1 1 1]
 [1 1 1]]
